## Search engine on unsupervised data

`This model can be used to answer by querying to make qa pair datasets`

In [1]:
path = ## Any pdf file, Here Attention is all you need research paper was used

In [2]:
from PyPDF2 import PdfReader

reader = PdfReader(path)
text  = ''
for pages in reader.pages:
    text += pages.extract_text()

In [5]:
len(text)

39472

In [24]:
import re

def preprocess(raw_string):
    #raw_string = raw_string.lower()
    text = re.sub(r'[^A-Za-z]+',' ',raw_string) # remove punctuations
    text = re.sub('\s+',' ',text)                   # remove whitespaces

    return text

clean_text = preprocess(text)

In [25]:
chunks = []

chunk_size = 300

for i in range(0, len(clean_text), chunk_size):
    chunk = text[i:i+chunk_size]
    chunks.append(chunk)
chunks[:5]

['Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parma',
 'r∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe ',
 'dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n',
 'based solely on attention mechanisms, dispensing with re

In [26]:
len(chunks)

120

In [27]:
from sentence_transformers import SentenceTransformer,util

model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_id)

In [28]:
embeds = model.encode(sentences=chunks)

In [84]:
def ST_search(corpus:list,query:str):
    corpus_embeddings = model.encode(corpus)
    query_embeddings = model.encode(query)
    similarity = util.cos_sim(corpus_embeddings,query_embeddings)
    
    index = torch.argmax(similarity)
    
    result_text = ''
    for i in range(index-1,index+1):
        result_text += corpus[i]
    print(f'Cosine Similarity = {similarity.max()}\n\nResult: {result_text}')
    return result_text

In [85]:
query = 'What is positional Encodings?'

In [86]:
result_text = ST_search(chunks,query)

Cosine Similarity = 0.6015917658805847

Result:  O(n)
Convolutional O(k·n·d2) O(1) O(logk(n))
Self-Attention (restricted) O(r·n·d) O(1) O(n/r)
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed. There are many choices of posit


In [35]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [i.split(' ') for i in chunks]
bm25 = BM25Okapi(tokenized_corpus)

query='What is Positional Encoding?'
doc_scores = bm25.get_scores(query.split(' '))

In [50]:
query_results = bm25.get_top_n(query.split(' '), chunks, n = 5)
user_index = chunks.index(query_results[0])

In [89]:

for i in range(49,51):
    result_text += chunks[i]

In [91]:
result_text

' O(n)\nConvolutional O(k·n·d2) O(1) O(logk(n))\nSelf-Attention (restricted) O(r·n·d) O(1) O(n/r)\n3.5 Positional Encoding\nSince our model contains no recurrence and no convolution, in order for the model to make use of the\norder of the sequence, we must inject some information about the relative or absolute position of the\ntokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the\nbottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel\nas the embeddings, so that the two can be summed. There are many choices of positayer types. nis the sequence length, dis the representation dimension, kis the kernel\nsize of convolutions and rthe size of the neighborhood in restricted self-attention.\nLayer Type Complexity per Layer Sequential Maximum Path Length\nOperations\nSelf-Attention O(n2·d) O(1) O(1)\nRecurrent O(n·d2) O(n) O(n)\nConvolutional O(k·n·d2) O(1) O(logk(n))\nSelf-Attention (restricted) O(r·n·d) O

In [95]:
import numpy as np

def bm25_search(corpus:list,query:str):
    tokenized_corpus = [i.split(' ') for i in chunks]
    bm25 = BM25Okapi(tokenized_corpus)
    doc_scores = bm25.get_scores(query.split(' '))
    user_index = chunks.index(query_results[0])
    
    # See below for reason of normalising
    i = np.argsort(doc_scores)  # indicies of sorted array in ascending order
    sorted_arr = doc_scores[i]  # Sorted array
    sorted_arr = sorted_arr[-10:]
    sorted_arr /= np.linalg.norm(sorted_arr)
    user_score = max(sorted_arr)
    
    result_text = ''
    for i in range(user_index-1,user_index+1):
        result_text += corpus[i]
    
    return user_score, result_text

user_score, result_text = bm25_search(chunks,query)
print(f'User_score : {user_score}\n\nAnswer : {result_text}')

User_score : 0.603451936669993

Answer : ayer types. nis the sequence length, dis the representation dimension, kis the kernel
size of convolutions and rthe size of the neighborhood in restricted self-attention.
Layer Type Complexity per Layer Sequential Maximum Path Length
Operations
Self-Attention O(n2·d) O(1) O(1)
Recurrent O(n·d2) O(n) O(n)
Convolutional O(k·n·d2) O(1) O(logk(n))
Self-Attention (restricted) O(r·n·d) O(1) O(n/r)
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or abs
